In [1]:
#get the dependencies that will be needed
import pandas as pd

In [2]:
#Load the raw data into the dataframes
school_data_raw = pd.read_csv("Resources/schools_complete.csv")
student_data_raw = pd.read_csv("Resources/students_complete.csv")

In [3]:
#Merged the dataframes together
school_data_complete = pd.merge(student_data_raw, school_data_raw, how="left", on=["school_name", "school_name"])

In [4]:
#Taking a look at the data. 
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [5]:
#using .unique() will give us the number of highschools in our data frame since there should not be any duplicate schools
number_of_schools = len(school_data_complete['school_name'].unique())

In [6]:
#For students we cannot assume there will not be duplicate names, but luckily each row is a unique student
number_of_students = len(school_data_complete['student_name'])

In [7]:
#To find the sum of the budget, we can just take the total from the budget column in the school dataframe
sum_budget = school_data_raw['budget'].sum()

In [14]:
#Finding the average score is simple using the .mean() function on the respective columns
average_math_score = school_data_complete["math_score"].mean()
average_reading_score = school_data_complete["reading_score"].mean()
average_overall_score = (average_math_score+average_reading_score)/2

In [9]:
#Now we need to find the percentage of passing students which is the total number of students with scores >70 / total students
passing_grade = 70
#First for reading
pass_reading = len(school_data_complete[school_data_complete['reading_score'] >= passing_grade])
pass_reading_per = pass_reading/number_of_students * 100
#Then for math
pass_math = len(school_data_complete[school_data_complete['math_score'] >= passing_grade])
pass_math_per = pass_math/number_of_students * 100
#Now it wants the average passing to be the average of the two averages
pass_overall_per = (pass_math_per + pass_reading_per)/2

In [16]:
#Next I am going to add all of these data points int a dictionary that will be turned into a dataframe
district_summary_dict = {"Total Schools":number_of_schools,"Total Students":number_of_students,"Total Budget":sum_budget,
                         "Average Math Score":average_math_score,"Average Reading Score":average_reading_score,
                         "% Passing Math":pass_math_per,"% Passing Reading":pass_reading_per,
                         "% Overall Passing Rate":average_overall_score}
district_summary_df = pd.DataFrame(district_summary_dict, index=[0])
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,80.431606


In [32]:
#For the School summary section I can use the raw school data as a starting point, i just need to make the School name the index
school_summary_df = school_data_raw.copy().set_index("school_name")
school_summary_drop_df = school_summary_df.drop(columns=["School ID"])
school_summary_drop_df.columns = ['School type', 'Total Students', 'Total School Budget']

In [63]:
#Now that I have the base dateframe, i should make a new copy and then add the column for per student budget
school_summary_base = school_summary_drop_df.copy()
school_summary_base["Per Student Budget"] = school_summary_base["Total School Budget"]/school_summary_base["Total Students"]
#school_summary_base.head()

In [64]:
#For these next few sections I should be able to use groupby to create a couple of series that i can add to my dataframe
math_group_df = school_data_complete['math_score'].groupby(school_data_complete['school_name']).mean()
reading_group_df = school_data_complete['reading_score'].groupby(school_data_complete['school_name']).mean()
#both of these series were pretty easy to get, but they are in alphabetical order. 
#The easy thing to do would be to sort the base df alphabetically then its a simple join
school_sum_alpha_sort = school_summary_base.sort_index(axis=0)
school_sum_scores = school_sum_alpha_sort.copy()
school_sum_scores['Average Math Score'] = math_group_df
school_sum_scores['Average Reading Score'] = reading_group_df
#school_sum_scores.head()

In [62]:
#the next part is similar to the previous but we first want to know how many students passed
math_pass_df = school_data_complete[school_data_complete["math_score"] >= 70].groupby(school_data_complete['school_name']).count()
reading_pass_df = school_data_complete[school_data_complete["reading_score"] >= 70].groupby(school_data_complete['school_name']).count()
#This returned a dataframe when i was expecting a series, but it should be fine for what i need
school_sum_pass = school_sum_scores.copy()
#in the following lines, math_score and reading_score are actually  variables of # of students that had a score >= 70
school_sum_pass["% Passing Math"] = math_pass_df["math_score"]/school_sum_pass["Total Students"]
school_sum_pass["% Passing Reading"] = reading_pass_df["reading_score"]/school_sum_pass["Total Students"]
school_sum_pass.head()

,School type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading
school_name,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,0.666801,0.819333
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,0.941335,0.970398
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,0.659885,0.807392
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,0.683096,0.792990
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,0.933924,0.971390


In [65]:
#Finally we add our last column with is just an average of the previous 2 columns
school_sum_overall = school_sum_pass.copy()
school_sum_overall["Overall Passing Rate"] = (school_sum_overall["% Passing Math"] + school_sum_overall["% Passing Reading"])/2
#school_sum_overall.head()

,School type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,0.666801,0.819333,0.743067
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,0.941335,0.970398,0.955867
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,0.659885,0.807392,0.733639
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,0.683096,0.792990,0.738043
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,0.933924,0.971390,0.952657


In [68]:
#Now that all the information is collected up, its time to display the top 5 performing schools
school_sum_top_overall = school_sum_overall.sort_values(by=["Overall Passing Rate"], ascending=False)
school_sum_top_overall.head()

,School type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,0.941335,0.970398,0.955867
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,0.932722,0.973089,0.952905
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,0.945946,0.959459,0.952703
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,0.933924,0.971390,0.952657
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,0.938677,0.965396,0.952037


In [69]:
#Then the bottom 5 schools
school_sum_bottom_overall = school_sum_overall.sort_values(by=["Overall Passing Rate"])
school_sum_bottom_overall.head()

,School type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,0.663666,0.802201,0.732933
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,0.659885,0.807392,0.733639
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,0.656839,0.813164,0.735002
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,0.660576,0.812224,0.736400
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,0.683096,0.792990,0.738043


In [109]:
#For this next part we want to breaks the scores up not only by school but by grade
#for this it wants us to break it down by creating a series for each grade, group the schools, then combine them
#first to make the dataframes that are filtered by grade
ninth_grade = student_data_raw[student_data_raw['grade'] == "9th"]
tenth_grade = student_data_raw[student_data_raw['grade'] == "10th"]
elfth_grade = student_data_raw[student_data_raw['grade'] == "11th"]
zwolfth_grade = student_data_raw[student_data_raw['grade'] == "12th"]
#Then to take all those filtered df's and group then by school and take the average
ninth_grade_school = ninth_grade.groupby(ninth_grade['school_name']).mean()
tenth_grade_school = tenth_grade.groupby(tenth_grade['school_name']).mean()
elfth_grade_school = elfth_grade.groupby(elfth_grade['school_name']).mean()
zwolfth_grade_school = zwolfth_grade.groupby(zwolfth_grade['school_name']).mean()
#Whats left are filtered, group'd by dataframes for each grade. now I need to put all the correct data into a new df
math_grade_summary = pd.DataFrame({"9th":ninth_grade_school['math_score'],
                                   "10th":tenth_grade_school['math_score'],
                                   "11th":elfth_grade_school['math_score'],
                                   "12th":zwolfth_grade_school['math_score']})
math_grade_summary

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [110]:
#Then to do the same thing but for reading
reading_grade_summary = pd.DataFrame({"9th":ninth_grade_school['reading_score'],
                                      "10th":tenth_grade_school['reading_score'],
                                      "11th":elfth_grade_school['reading_score'],
                                      "12th":zwolfth_grade_school['reading_score']})
reading_grade_summary

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [28]:
school_data_raw.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [17]:
student_data_raw.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84
